<h1>predict survival on the titanic - kaggle competition</h1>

In [117]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import keras
import seaborn as sns
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
sns.set()

<b>Exclude columns with a high percentage of missing values while importing data and choosing features.</b>

In [97]:
df = pd.read_csv("train.csv")
X = df.drop(["Name","Cabin","Ticket","Survived","PassengerId"],axis=1)
Y = df["Survived"]

<b>one hot encoding the categorical variables</b>

In [98]:
categorical_cols = ["Sex","Pclass","Embarked"]
X = pd.get_dummies(X,columns=categorical_cols,drop_first=True)

<b>normalizing nomerical variables</b>

In [99]:
numerical_cols = X[["SibSp","Parch","Age","Fare"]]
numerical_cols = (numerical_cols - numerical_cols.mean())/numerical_cols.std()
X[["SibSp","Parch","Age","Fare"]] = numerical_cols

<b>20% of passengers age are missing but as we guess age is critical information in predicting survival In order to test our assumption that age is a key factor in predicting survival, we first analyse the relationship between survival and age. Then, we attempt to develop a model to estimate missing ages based on other factors.</b>  

In [100]:
df[["Survived","Age"]].corr()

,Survived,Age
Survived,1.000000,-0.077221
Age,-0.077221,1.000000


<b>here we used pretrained KNN model to predict missed ages</b>

In [101]:
#predict missed ages
X_missed_age = X[df.Age.isnull()].copy()
X_missed_age = X_missed_age[["SibSp","Parch","Sex_male","Pclass_2","Pclass_3"]]
age_prediction_model = pickle.load(open("KNNRegressor_ageprediction_titanic.sav","rb"))
X.Age[X.Age.isnull()] = age_prediction_model.predict(X_missed_age)

/tmp/ipykernel_21664/386599295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[X.Age.isnull()] = age_prediction_model.predict(X_missed_age)


<b>split dataset to train and test</b>

In [102]:
X_train , X_test , y_train , y_test = train_test_split(X,Y,train_size=0.8)
X_train , X_val , y_train , y_val = train_test_split(X_train,y_train,train_size=0.8)

<b>first we try a simple model for prediction</b>

In [139]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [140]:
accuracy_score(y_train,model.predict(X_train))

0.8629173989455184

In [141]:
accuracy_score(y_val,model.predict(X_val))

0.8041958041958042

<b>In this data set, only 38% of passengers survived, therefore using a basic model that predicts each passenger as dead, we achieve a higher than 50% accuracy, hence it's preferable to use recall score when evaluating models.</b>

In [142]:
recall_score(y_train,model.predict(X_train))

0.8185840707964602

In [143]:
recall_score(y_val,model.predict(X_val))

0.7592592592592593

<b>now,we try neural networks to see how far we can get</b>

In [68]:
modelNN = keras.Sequential(
    [
        keras.Input(shape=(9,)),
        layers.Dense(units=32,activation="relu",kernel_regularizer=keras.regularizers.L2()),
        layers.Dense(units=32,activation="relu",kernel_regularizer=keras.regularizers.L2()),
        layers.Dense(units=1,activation="sigmoid"),
    ]
)

In [146]:
modelNN.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy","Recall"])
modelNN.fit(X_train,y_train,batch_size=569,epochs=256,validation_data=(X_val,y_val))

Epoch 1/256
1/1 [==============================] - 1s 1s/step - loss: 0.5328 - accuracy: 0.8155 - recall: 0.6637 - val_loss: 0.4655 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 2/256
1/1 [==============================] - 0s 63ms/step - loss: 0.5353 - accuracy: 0.8137 - recall: 0.7035 - val_loss: 0.4553 - val_accuracy: 0.8741 - val_recall: 0.7407
Epoch 3/256
1/1 [==============================] - 0s 63ms/step - loss: 0.5300 - accuracy: 0.8207 - recall: 0.6858 - val_loss: 0.4468 - val_accuracy: 0.8811 - val_recall: 0.7407
Epoch 4/256
1/1 [==============================] - 0s 71ms/step - loss: 0.5287 - accuracy: 0.8207 - recall: 0.6770 - val_loss: 0.4441 - val_accuracy: 0.8811 - val_recall: 0.7407
Epoch 5/256
1/1 [==============================] - 0s 41ms/step - loss: 0.5292 - accuracy: 0.8225 - recall: 0.6814 - val_loss: 0.4446 - val_accuracy: 0.8811 - val_recall: 0.7407
Epoch 6/256
1/1 [==============================] - 0s 45ms/step - loss: 0.5275 - accuracy: 0.8190 - recall: 0.68

Epoch 47/256
1/1 [==============================] - 0s 65ms/step - loss: 0.4949 - accuracy: 0.8260 - recall: 0.7035 - val_loss: 0.4291 - val_accuracy: 0.8392 - val_recall: 0.7222
Epoch 48/256
1/1 [==============================] - 0s 38ms/step - loss: 0.4943 - accuracy: 0.8260 - recall: 0.7035 - val_loss: 0.4290 - val_accuracy: 0.8392 - val_recall: 0.7222
Epoch 49/256
1/1 [==============================] - 0s 43ms/step - loss: 0.4938 - accuracy: 0.8260 - recall: 0.7035 - val_loss: 0.4288 - val_accuracy: 0.8392 - val_recall: 0.7222
Epoch 50/256
1/1 [==============================] - 0s 33ms/step - loss: 0.4933 - accuracy: 0.8260 - recall: 0.7035 - val_loss: 0.4281 - val_accuracy: 0.8392 - val_recall: 0.7222
Epoch 51/256
1/1 [==============================] - 0s 38ms/step - loss: 0.4928 - accuracy: 0.8278 - recall: 0.7080 - val_loss: 0.4272 - val_accuracy: 0.8392 - val_recall: 0.7222
Epoch 52/256
1/1 [==============================] - 0s 51ms/step - loss: 0.4923 - accuracy: 0.8278 - reca

Epoch 93/256
1/1 [==============================] - 0s 64ms/step - loss: 0.4754 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4113 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 94/256
1/1 [==============================] - 0s 60ms/step - loss: 0.4750 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4110 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 95/256
1/1 [==============================] - 0s 60ms/step - loss: 0.4747 - accuracy: 0.8348 - recall: 0.7301 - val_loss: 0.4106 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 96/256
1/1 [==============================] - 0s 53ms/step - loss: 0.4743 - accuracy: 0.8348 - recall: 0.7301 - val_loss: 0.4105 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 97/256
1/1 [==============================] - 0s 45ms/step - loss: 0.4740 - accuracy: 0.8348 - recall: 0.7301 - val_loss: 0.4105 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 98/256
1/1 [==============================] - 0s 39ms/step - loss: 0.4736 - accuracy: 0.8348 - reca

1/1 [==============================] - 0s 44ms/step - loss: 0.4623 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4038 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 139/256
1/1 [==============================] - 0s 51ms/step - loss: 0.4621 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4036 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 140/256
1/1 [==============================] - 0s 47ms/step - loss: 0.4618 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4037 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 141/256
1/1 [==============================] - 0s 44ms/step - loss: 0.4616 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4031 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 142/256
1/1 [==============================] - 0s 48ms/step - loss: 0.4614 - accuracy: 0.8330 - recall: 0.7257 - val_loss: 0.4030 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 143/256
1/1 [==============================] - 0s 43ms/step - loss: 0.4611 - accuracy: 0.8330 - recall: 0.72

1/1 [==============================] - 0s 48ms/step - loss: 0.4530 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3980 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 184/256
1/1 [==============================] - 0s 43ms/step - loss: 0.4529 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3978 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 185/256
1/1 [==============================] - 0s 47ms/step - loss: 0.4527 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3975 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 186/256
1/1 [==============================] - 0s 45ms/step - loss: 0.4526 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3978 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 187/256
1/1 [==============================] - 0s 48ms/step - loss: 0.4524 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3975 - val_accuracy: 0.8601 - val_recall: 0.7407
Epoch 188/256
1/1 [==============================] - 0s 42ms/step - loss: 0.4522 - accuracy: 0.8330 - recall: 0.73

1/1 [==============================] - 0s 35ms/step - loss: 0.4469 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3952 - val_accuracy: 0.8531 - val_recall: 0.7222
Epoch 229/256
1/1 [==============================] - 0s 42ms/step - loss: 0.4468 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3947 - val_accuracy: 0.8531 - val_recall: 0.7222
Epoch 230/256
1/1 [==============================] - 0s 44ms/step - loss: 0.4467 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3953 - val_accuracy: 0.8531 - val_recall: 0.7222
Epoch 231/256
1/1 [==============================] - 0s 42ms/step - loss: 0.4466 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3943 - val_accuracy: 0.8531 - val_recall: 0.7222
Epoch 232/256
1/1 [==============================] - 0s 45ms/step - loss: 0.4465 - accuracy: 0.8330 - recall: 0.7301 - val_loss: 0.3949 - val_accuracy: 0.8531 - val_recall: 0.7222
Epoch 233/256
1/1 [==============================] - 0s 44ms/step - loss: 0.4464 - accuracy: 0.8330 - recall: 0.73

In [154]:
accuracy_score(y_test,modelNN.predict(X_test)>0.7)

6/6 [==============================] - 0s 2ms/step


0.8324022346368715